In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from zipfile import ZipFile
import os
import random

zip_path = "/content/drive/MyDrive/9517/archive.zip"
extract_path = "/content/skyview_data"

with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

base_path = os.path.join(extract_path, "Aerial_Landscapes")
classes = sorted(os.listdir(base_path))
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
print("类别数量:", len(classes))
print("类别示例:", classes[:5])

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def collect_image_paths(base_path):
    train_paths, test_paths = [], []
    train_labels, test_labels = [], []
    for cls_name in sorted(os.listdir(base_path)):
        cls_folder = os.path.join(base_path, cls_name)
        if os.path.isdir(cls_folder):
            all_imgs = sorted([os.path.join(cls_folder, f) for f in os.listdir(cls_folder) if f.endswith('.jpg') or f.endswith('.png')])
            random.shuffle(all_imgs)
            n_total = len(all_imgs)
            n_train = int(0.8 * n_total)
            train_paths += all_imgs[:n_train]
            train_labels += [class_to_idx[cls_name]] * n_train
            test_paths += all_imgs[n_train:]
            test_labels += [class_to_idx[cls_name]] * (n_total - n_train)
    return train_paths, train_labels, test_paths, test_labels

train_paths, train_labels, test_paths, test_labels = collect_image_paths(base_path)
print(f"训练图像数量: {len(train_paths)}，测试图像数量: {len(test_paths)}")

In [ ]:
def extract_sift_from_paths(image_paths, max_per_image=100):
    sift = cv2.SIFT_create()
    descriptors = []
    for path in tqdm(image_paths):
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, des = sift.detectAndCompute(gray, None)
        if des is not None and len(des) > 0:
            if max_per_image and des.shape[0] > max_per_image:
                idx = np.random.choice(des.shape[0], max_per_image, replace=False)
                des = des[idx]
            descriptors.append(des)
    return descriptors

In [ ]:
print("提取训练集部分特征以训练词典...")
sampled_descriptors = extract_sift_from_paths(train_paths, max_per_image=100)
all_sampled = np.vstack(sampled_descriptors)
print("开始训练 KMeans...")
kmeans = KMeans(n_clusters=150, random_state=42, n_init='auto')
kmeans.fit(all_sampled)

In [ ]:
def compute_bow_feature(image_path, kmeans):
    sift = cv2.SIFT_create()
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, des = sift.detectAndCompute(gray, None)
    hist = np.zeros(kmeans.n_clusters)
    if des is not None and len(des) > 0:
        words = kmeans.predict(des)
        for w in words:
            hist[w] += 1
    return hist

In [ ]:
X_train = [compute_bow_feature(path, kmeans) for path in tqdm(train_paths)]
X_test = [compute_bow_feature(path, kmeans) for path in tqdm(test_paths)]
y_train = train_labels
y_test = test_labels

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = SVC(kernel='linear', C=1.0, random_state=42)
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
report = classification_report(y_test, y_pred, target_names=classes)

print(f"准确率 (Accuracy): {accuracy:.4f}")
print(f"精确率 (Precision): {precision:.4f}")
print(f"召回率 (Recall): {recall:.4f}")
print(f"F1 分数 (F1-score): {f1:.4f}")
print("\n分类报告:\n", report)

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
plt.title("混淆矩阵 (Confusion Matrix)")
plt.xlabel("预测标签 (Predicted)")
plt.ylabel("真实标签 (Actual)")
plt.tight_layout()
plt.show()